# core

> Fill in a module description here

In [ ]:
# | default_exp core

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

In [ ]:
# | export
from dataclasses import dataclass
from abc import ABC, abstractmethod

from exex.imports import *
from exex.utils import *

In [ ]:
# | export
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

In [ ]:
u.meter

meter

### Unit

In [ ]:
# #| export
# @dataclass
# class Unit:
#     """
#     Default Units
#     """

#     # SI Unit
#     LENGTH = 'meter'
#     MASS = 'kilogram'
#     TIME = 'second'
#     TEMPERATURE = 'kelvin'

#     # Derived from SI Unit
#     MOLAR_MASS = 'grams / mole'
#     MOLE = 'mole'
#     SPECIFIC_HEAT = 'joule / (kilogram kelvin)'
#     PRESSURE = 'pascal'
#     VOLUME = 'liter'

In [ ]:
# | export
class Unit:
    """
    Default SI Units
    """

    LENGTH = u.meter
    MASS = u.kilogram
    TIME = u.second
    TEMPERATURE = u.kelvin

    """
    Derived from SI Units
    """
    MOLAR_MASS = u.gram / u.mole
    MOLE = u.mole
    SPECIFIC_HEAT = u.joule / (u.kilogram * u.kelvin)
    PRESSURE = u.pascal
    VOLUME = u.liter

In [ ]:
# | export
def unit2expr(unit):
    pass

In [ ]:
Unit.MASS

kilogram

In [ ]:
Unit.SPECIFIC_HEAT

joule/(kelvin*kilogram)

In [ ]:
# | hide
test_eq(Unit.MASS, u.kilogram)
test_eq(str(Unit.MASS), "kilogram")

In [ ]:
# | export
ureg = pint.UnitRegistry(system="SI")
Q = ureg.Quantity  # quantity

In [ ]:
@patch
def to_standard(self: Q):
    return 1

In [ ]:
# moles = Q(0.214, Unit.MOLE)

In [ ]:
# moles

In [ ]:
# moles.__dict__

In [ ]:
# moles.dimensionality

In [ ]:
# #| hide
# test_eq(moles.magnitude, 0.214)
# test_eq(moles.units, "mole")

In [ ]:
# specific_heat = Q(0.235, Unit.SPECIFIC_HEAT)

In [ ]:
# specific_heat.ito_reduced_units()

In [ ]:
# #| hide
# test_eq(specific_heat.magnitude, 0.235)
# test_eq(specific_heat.units, 'joule/(kelvin kilogram)')

In [ ]:
# m = Q('3.21 kilogram')

In [ ]:
# m

In [ ]:
@ureg.check("[length]")
def foo(length):
    return length

In [ ]:
foo("10 meter")

'10 meter'

In [ ]:
@ureg.wraps((ureg.meter, ureg.gram), (ureg.meter, ureg.kilogram))
def mypp(length, mass):
    return length, mass

In [ ]:
mypp("10 kilometer", "1 kilogram")

(10000.0 <Unit('meter')>, 1 <Unit('gram')>)

In [ ]:
@ureg.check("[mass]")
@ureg.wraps(None, None)
def set_mass(mass):
    return mass

In [ ]:
set_mass("20 kilogram")

'20 kilogram'

In [ ]:
@ureg.check("[mass]")
@ureg.wraps(ureg.gram, ureg.kilogram)
def set_mass(mass):
    return mass

In [ ]:
set_mass("20 kilogram")

20 <Unit('gram')>

In [ ]:
set_mass("20 grams")

0.02 <Unit('gram')>

### Data

In [ ]:
# | export
class Object:
    pass

In [ ]:
# | export
class PropertyData(dict):
    pass

In [ ]:
c = PropertyData({"0": {"mass": 2, "object": Object()}})

In [ ]:
c["0"]

{'mass': 2, 'object': <__main__.Object>}

In [ ]:
c["1"] = {"mass": 2.1, "object": "XXX"}

### Proprety

In [ ]:
# | export
@docs
class BaseProperty:
    def __init__(self, cmp):  # chemical substance
        self.is_constant: bool = False
        self.compound = cmp
        self.cmp = cmp
        self._data = PropertyData()

        self._connections = []
        self.laws = dict()
        self.func_changed = Event()

    @property
    def name(self) -> str:
        return camel_to_snake(self.__class__.__name__)

    @classmethod
    @property
    def snake_name(cls) -> str:  # return the snake style name
        return camel_to_snake(cls.__name__)

    def expr(self, t: int):  # time
        return self.symbol(t)

    def __call__(self, t: int, eval: bool = False, **kwargs):  # time
        self.t = t
        self.kwargs = {**kwargs, "eval": eval}

        expr = self.expr(t)

        if eval == True:
            return self.eval(expr, t)
        else:
            return expr

    def add_law(self, law):
        if not law in self.laws:
            self.laws[camel_to_snake(law.__class__.__name__)] = law

    _docs = dict(cls_doc="Property", add_law="", expr="Symbolic expression")

In [ ]:
# | export
@patch
def symbol(self: BaseProperty, t):  # symbolic expression of the property
    """Rewrite this method if you want to modify"""
    return smp.symbols(f"{self.abbrv}_{self.cmp.snake_name}-{t}", real=True)

In [ ]:
# | export
@patch
def set_val(self: BaseProperty, val: str, t: int):
    self._data[t] = {"val": val}

In [ ]:
# | export
@patch
def get_val(self: BaseProperty, t: int):  # time
    if self.is_constant is True:
        return self.compute()
    else:
        return self._data[t]["val"] if t in self._data else self.symbol(t)

In [ ]:
# | export
@patch
def eval(self: BaseProperty, expr, t: int):  # express  # time
    return expr.xreplace({expr: self.get_val(t=t)})

In [ ]:
# | export
@patch
def is_empty(self: BaseProperty, t):
    return type(self.get_val(t))
    # return True if isinstance(type(self.get_val(t)), sympy.core.symbol.Symbol) else False

In [ ]:
# | export
@docs
class Property(BaseProperty):
    _docs = dict(cls_doc="Property that varies in time")

#### Constant Property

In [ ]:
# | export
@docs
class ConstantProperty(BaseProperty):
    @abstractmethod
    def compute(self):
        pass

    _docs = dict(
        cls_doc="Property that invariant in time", compute="Calculate the value"
    )

##### Property

In [ ]:
# | export
class Property(BaseProperty):
    pass

In [ ]:
# | export
class PropertyObservable(Property):
    pass

##### Molar Mass

Molar Mass is

In [ ]:
# | export
class Mass(Property):
    def __init__(self, compound):
        super().__init__(compound)
        self.abbrv = "m"
        self.unit = Unit.MASS

In [ ]:
# | export
class MolarMass(ConstantProperty):
    def __init__(self, compound):
        self.abbrv = "M"
        self.unit = Unit.MOLAR_MASS
        super().__init__(compound)
        self.is_constant = True

    def compute(self):
        mass = 0
        for element in self.compound.elements:
            mass += element.AtomicMass

        return mass * self.unit

In [ ]:
# | export
class Mole(Property):
    def __init__(self, compound):
        super().__init__(compound)
        self.abbrv = "n"
        self.unit = Unit.MOLE

In [ ]:
# | export
class Pressure(Property):
    def __init__(self, compound):
        super().__init__(compound)
        self.abbrv = "P"
        self.unit = Unit.PRESSURE

In [ ]:
# | export
class Volume(Property):
    def __init__(self, compound):
        super().__init__(compound)
        self.abbrv = "V"
        self.unit = Unit.VOLUME

In [ ]:
# | export
class Temperature(Property):
    def __init__(self, compound):
        super().__init__(compound)
        self.abbrv = "T"
        self.unit = Unit.TEMPERATURE

### Law

In [ ]:
# | export
class BaseLaw(ABC):
    @property
    def name(self) -> str:
        return camel_to_snake(self.__class__.__name__)

    @classmethod
    @property
    def snake_name(cls) -> str:  # return the snake style name
        return camel_to_snake(cls.__name__)

    def n_known_variables(self, timestep: int) -> int:  # the number of known variables
        n_knowns = 0
        for p in self.properties:
            name = camel_to_snake(p["object"].__name__)
            if timestep in self.compound.properties[name]._data:
                n_knowns += 1
        return n_knowns

    def is_solveable(self, timestep: int) -> bool:  # the timestep
        n_unknowns = len(self.properties)
        n_knowns = self.n_known_variables(timestep)
        print(f"n_unknowns={n_unknowns} and n_knowns={n_knowns}")
        return n_unknowns - n_knowns <= 1

    def _run_config(self) -> None:  # run all configuration
        self._config_properties()

    def _config_properties(self) -> None:  # add law's properties to compound

        for p in self.properties:
            # name = camel_to_snake(p['object'].__name__)
            name = p["object"].snake_name

            if not name in self.compound.properties:
                self.compound.properties[name] = p["object"](compound=self.compound)

            self.compound.properties[name].add_law(self)

    def solve(
        self,
        timestep: int,  # the time step of the unknown variable
        unknown: str,  # the unknown variable that you want to solve
    ):
        unknown_symbol = self.compound.properties[unknown].symbol
        return smp.solve(self.expression, unknown_symbol)

    def __call__(self, *args, **kwargs):
        return self.expr(*args, **kwargs)

    @abstractmethod
    def expr(self):
        """The symbolic expression of the law"""
        pass

    def __repr__(self):
        return f"Law({self.name})"

In [ ]:
# | export
class Law(BaseLaw):
    pass

### System

System is an mediator that compounds and reactions communicate through each others.

In [ ]:
# #| export
# class System:
#     def __init__(self):
#         self.reactions = dict()
#         self.universe = None
#         self.subscribers: dict[str, Callable] = dict()
#         self.current_time: int = None
#         self.highest_time: int = None